In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow.keras import layers

In [0]:
dataset, info = tfds.load(name='horses_or_humans',with_info=True, as_supervised=True)

In [0]:
train_dataset, test_dataset = dataset['train'], dataset['test']
num_train = info.splits['train'].num_examples
num_test = info.splits['test'].num_examples

In [0]:
#normalizing and resizing the image to make it fit for pre-trained model.
def normalize(image, labels):
  image = tf.image.resize(image, (128, 128))/255.0
  return image, labels


BATCH_SIZE = 64
#shuffling and implementing normalize function on train, test and validation dataset
train_batches     = train_dataset.shuffle(num_train//4).map(normalize).batch(BATCH_SIZE).prefetch(1)
validation_batches      = test_dataset.map(normalize).batch(BATCH_SIZE).prefetch(1)
test_batches     = test_dataset.map(normalize).batch(3000).prefetch(1)

In [0]:
#downloading the pre-trained model
feature_extractor2 = hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_035_128/feature_vector/4",input_shape=(128, 128, 3)
                                   )

In [0]:
#creating model
model = tf.keras.Sequential([feature_extractor2,
                            layers.Dense(1, activation='sigmoid')])

In [0]:
#training the model
model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy'])

EPOCHS = 6
history = model.fit(train_batches,
                    epochs=EPOCHS,
                   validation_data=validation_batches)

Epoch 1/6
17/17 [==============================] - 10s 594ms/step - loss: 0.3582 - accuracy: 0.8627 - val_loss: 0.1612 - val_accuracy: 0.9805
Epoch 2/6
17/17 [==============================] - 5s 294ms/step - loss: 0.0801 - accuracy: 0.9893 - val_loss: 0.0851 - val_accuracy: 0.9883
Epoch 3/6
17/17 [==============================] - 5s 287ms/step - loss: 0.0421 - accuracy: 0.9942 - val_loss: 0.0686 - val_accuracy: 0.9961
Epoch 4/6
17/17 [==============================] - 5s 298ms/step - loss: 0.0300 - accuracy: 0.9971 - val_loss: 0.0593 - val_accuracy: 0.9961
Epoch 5/6
17/17 [==============================] - 5s 291ms/step - loss: 0.0237 - accuracy: 0.9981 - val_loss: 0.0527 - val_accuracy: 0.9961
Epoch 6/6
17/17 [==============================] - 5s 294ms/step - loss: 0.0197 - accuracy: 0.9990 - val_loss: 0.0484 - val_accuracy: 0.9961


In [0]:
model.save('/content/model.h5')

In [0]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
open ("model.tflite" , "wb") .write(tfmodel)


1604524